<a target="_blank" href="https://colab.research.google.com/github/chasubeen/Kwater/tree/subeen">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

import os
import chardet

In [3]:
pd.set_option('display.float_format', '{:.6f}'.format) # 과학적 표기법 x

# **1. 데이터 병합**

In [4]:
### 파일 병합을 위한 함수

def file_concat(path, file_list):
  dfs = []  # DataFrame들을 저장할 리스트

  for file in file_list:
    # 파일 확장자가 .csv인 경우에만 처리
    if file.endswith(".csv"):
      file_path = os.path.join(path, file)

      # 파일을 열어서 인코딩 확인
      with open(file_path, 'rb') as f:
        rawdata = f.read()
      result = chardet.detect(rawdata)
      enc = result['encoding']

      # 해당 파일의 인코딩에 맞게 열어서 DataFrame으로 변환
      df = pd.read_csv(file_path, encoding = enc)
      dfs.append(df)

  # DataFrame들을 세로 방향으로 이어붙임
  df = pd.concat(dfs, axis = 0, ignore_index = True)  # ignore_index = True로 해줘야 오류 없이 밑으로 결합됨

  return df

## **1-1. 저수량 데이터**
- 각 댐에 대한 정보를 포함하고 있는 데이터    
- 활용 데이터 목록
  - [다목적댐 운영 정보(일자료)](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b77fd770-38bb-11ea-be28-4fa0eb812a46)
  - [MyWater 물정보포털수문자료](https://www.water.or.kr/kor/realtime/sumun/index.do?mode=sumun&menuId=13_91_93_94)

### **📌 Data Description**
1. dam_nm: 댐이름
2. obsr_de: 일자
3. rain_qy: 강우량
4. inflow_qy: 유입량
5. tot_dcwtr_qy: 방류량
6. rsvwt_qy: 저수량
7. dam_rsvwt_rt: 저수율

### **a) 데이터 불러오기**

In [7]:
water_path = '/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/다목적댐운영정보' # path 변수에 경로 지정

In [8]:
water_list = os.listdir(water_path)
print(water_list)
print(len(water_list))

['201901.csv', '201902.csv', '201903.csv', '201904.csv', '201905.csv', '201906.csv', '201907.csv', '201908.csv', '201909.csv', '201910.csv', '201912.csv', '202001.csv', '202002.csv', '202003.csv', '202004.csv', '202005.csv', '202006.csv', '202007.csv', '202008.csv', '202101.csv', '202102.csv', '202103.csv', '202104.csv', '202105.csv', '202106.csv', '202107.csv', '202108.csv', '202109.csv', '202110.csv', '202111.csv', '202112.csv', '202201.csv', '202204.csv', '202205.csv', '202206.csv', '202207.csv', '202209.csv', '201911.csv', '202012.csv', '202202.csv', '202009.csv', '202010.csv', '202203.csv', '202011.csv', '202208.csv', '202211.csv', '202212.csv', '202210.csv']
48


- 4개년치에 해당하는 48개 달에 대한 데이터가 정상적으로 load됨

In [9]:
df = file_concat(water_path, water_list)

In [10]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,20190101,0.000000,35.381000,104.637000,2147.765000,78.100000
1,횡성,20190101,0.000000,0.373000,1.614000,71.945000,82.800000
2,소양강,20190101,0.000000,9.063000,14.959000,1864.739000,64.300000
3,안동,20190101,0.000000,1.122000,16.978000,929.947000,74.500000
4,임하,20190101,0.000000,0.000000,8.555000,374.233000,62.900000
...,...,...,...,...,...,...,...
30676,횡성,20221027,0.000000,1.265000,1.265000,71.998300,82.850000
30677,횡성,20221028,0.000000,1.307000,1.307000,71.998300,82.900000
30678,횡성,20221029,0.000000,1.434000,1.434000,71.998300,82.850000
30679,횡성,20221030,0.000000,1.369000,1.369000,71.998300,82.850000


- 365(일) * 4(년) * 21(댐 개수) = 30660
- 2020년의 경우 2월 29일까지 존재 -> 21개 추가

In [11]:
### 데이터 정보 확인

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dam_nm        30681 non-null  object 
 1   obsr_de       30681 non-null  int64  
 2   rain_qy       30681 non-null  object 
 3   inflow_qy     30681 non-null  float64
 4   tot_dcwtr_qy  30681 non-null  float64
 5   rsvwt_qy      30681 non-null  float64
 6   dam_rsvwt_rt  30681 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.6+ MB


- obsr_de(관측일)과 rain_qy(강우량)의 데이터형(dtype)이 잘못 설정되어 있는 것 같음
  - obsr_de: object -> datetime
  - rain_qy: object -> float64

In [12]:
# 결측치나 변경할 수 없는 값이 있다면 일단 모두 무시하고 데이터형 변경

df['obsr_de'] = df['obsr_de'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d', errors = 'coerce'))
df['rain_qy'] = df['rain_qy'].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))

In [13]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.000000,35.381000,104.637000,2147.765000,78.100000
1,횡성,2019-01-01,0.000000,0.373000,1.614000,71.945000,82.800000
2,소양강,2019-01-01,0.000000,9.063000,14.959000,1864.739000,64.300000
3,안동,2019-01-01,0.000000,1.122000,16.978000,929.947000,74.500000
4,임하,2019-01-01,0.000000,0.000000,8.555000,374.233000,62.900000


In [14]:
df['dam_nm'].unique()

array(['충주', '횡성', '소양강', '안동', '임하', '성덕', '영주', '군위', '김천부항', '보현산',
       '합천', '남강', '밀양', '용담', '대청', '보령', '부안', '섬진강', '주암(본)', '주암(조)',
       '장흥'], dtype=object)

- 21개 댐에 대한 정보를 포함하고 있음

### **b) 이상치 처리**
- 저수량 데이터 특성상 값들 사이의 편차가 크다.
  - 이상치를 걸러내는 것에 의미가 없을 것으로 판단되어 별도로 진행 x

### **c) 결측치 처리**

In [15]:
# 현재 데이터 내의 결측치는 공백으로 처리되어 있음
# 데이터프레임 내의 공백을 NaN으로 변경

df.replace('', np.nan, inplace = True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dam_nm        30681 non-null  object        
 1   obsr_de       30681 non-null  datetime64[ns]
 2   rain_qy       30680 non-null  float64       
 3   inflow_qy     30681 non-null  float64       
 4   tot_dcwtr_qy  30681 non-null  float64       
 5   rsvwt_qy      30681 non-null  float64       
 6   dam_rsvwt_rt  30681 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.6+ MB


- rain_qy(강수량)에 결측치가 존재

In [17]:
df.loc[df['rain_qy'].isna(),:]

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
659,김천부항,2019-02-01,NaN,0.026664,1.616854,37.143000,68.400000


In [18]:
# 해당 댐의 해당 연도 + 달의 강수량 최빈값으로 대체

df['rain_qy'] = df.groupby(['dam_nm', df['obsr_de'].dt.year ,df['obsr_de'].dt.month])['rain_qy'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [19]:
df.loc[df['rain_qy'].isna(),:]

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt


- 결측치가 제대로 채워짐

In [20]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.000000,35.381000,104.637000,2147.765000,78.100000
1,횡성,2019-01-01,0.000000,0.373000,1.614000,71.945000,82.800000
2,소양강,2019-01-01,0.000000,9.063000,14.959000,1864.739000,64.300000
3,안동,2019-01-01,0.000000,1.122000,16.978000,929.947000,74.500000
4,임하,2019-01-01,0.000000,0.000000,8.555000,374.233000,62.900000


### **d) 단위 변경**
- 유입량, 방류량을 1일당 총량으로 변경
  - m3/s 단위를 m3로 변경
- 하루 기준: 60초 * 60분 * 24시간 = 86400초
- 총저수량의 경우 m3 * 백만을 m3으로 변경

In [21]:
df['inflow_qy'] = df['inflow_qy'] * 86400
df['tot_dcwtr_qy'] = df['tot_dcwtr_qy'] * 86400

In [22]:
df['rsvwt_qy'] = df['rsvwt_qy'] * 1000000

In [23]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.000000,3056918.400000,9040636.800000,2147765000.000000,78.100000
1,횡성,2019-01-01,0.000000,32227.200000,139449.600000,71945000.000000,82.800000
2,소양강,2019-01-01,0.000000,783043.200000,1292457.600000,1864739000.000000,64.300000
3,안동,2019-01-01,0.000000,96940.800000,1466899.200000,929947000.000000,74.500000
4,임하,2019-01-01,0.000000,0.000000,739152.000000,374233000.000000,62.900000


In [24]:
duplicates = df[df.duplicated(subset = ['dam_nm', 'obsr_de'], keep = False)]
print(duplicates)

Empty DataFrame
Columns: [dam_nm, obsr_de, rain_qy, inflow_qy, tot_dcwtr_qy, rsvwt_qy, dam_rsvwt_rt]
Index: []


- 중복된 데이터는 x

## **1-2. 제원정보 데이터**
- 21개의 댐에 대한 제원 정보를 포함하는 데이터
  - 높이, 길이, 수위, 용량 등
- 해당 데이터에서 필요한 컬럼만을 선택하여 저장
  - 댐 이름, 유효저수량
- 활용 데이터 목록
  - [다목적댐 제원 정보](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b74c6b60-38bb-11ea-be28-4fa0eb812a46)


### **📌Data Description**
1. dam_nm: 댐이름
2. valid_rsvwt_cpcty_qy: 유효저수용량

In [25]:
# 데이터 불러오기

df_valid_capa = pd.read_csv('/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/다목적댐_제원_정보.csv', sep = ',')
df_valid_capa

,dam_nm,valid_rsvwt_cpcty_qy
0,군위,40.100000
1,김천부항,42.600000
2,남강,299.700000
3,대청,790.000000
4,밀양,69.800000
5,보령,108.700000
6,보현산,17.880000
7,부안,35.600000
8,섬진강,429.000000
9,성덕,24.800000


In [26]:
df_valid_capa['dam_nm'].unique()

array(['군위', '김천부항', '남강', '대청', '밀양', '보령', '보현산', '부안', '섬진강', '성덕',
       '소양강', '안동', '영주', '용담', '임하', '장흥', '주암(본)', '주암(조)', '충주', '합천',
       '횡성'], dtype=object)

- 정상적으로 **21개**의 댐의 유효저수량이 포함되어 있음

In [27]:
# 두 데이터가 가진 댐이 서로 동일한지 확인

sum(~df['dam_nm'].isin(df_valid_capa['dam_nm']))

0

- 두 개의 데이터에서 동일한 댐에 대한 정보들을 가지고 있음을 확인할 수 있다.
  - 댐 이름에서 차이가 x

In [28]:
# 기존 df 와 df_valid_capa 병합하기(merge)

df = pd.merge(left = df, right = df_valid_capa,
              how = "inner", on = "dam_nm") # dam_nm 기준으로 결합

In [29]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt,valid_rsvwt_cpcty_qy
0,충주,2019-01-01,0.000000,3056918.400000,9040636.800000,2147765000.000000,78.100000,1789.000000
1,충주,2019-01-02,0.000000,2209852.800000,9036748.800000,2140938000.000000,77.900000,1789.000000
2,충주,2019-01-03,0.000000,1373932.800000,9039427.200000,2133272000.000000,77.600000,1789.000000
3,충주,2019-01-04,0.000000,2237846.400000,9038476.800000,2126472000.000000,77.300000,1789.000000
4,충주,2019-01-05,0.000000,2248300.800000,9036576.000000,2119683000.000000,77.100000,1789.000000
...,...,...,...,...,...,...,...,...
30676,장흥,2022-10-27,0.000000,0.000000,181267.200000,78686600.000000,41.200000,171.000000
30677,장흥,2022-10-28,0.000000,0.000000,181612.800000,78490900.000000,41.090000,171.000000
30678,장흥,2022-10-29,0.000000,0.000000,174441.600000,78295400.000000,40.990000,171.000000
30679,장흥,2022-10-30,0.000000,41990.400000,172195.200000,78165200.000000,40.920000,171.000000


- 정확히 결합된 것을 확인할 수 있음

## **1-3. 기상자료**
- 각 댐이 위치하는 지역의 기상정보에 대한 데이터
  - 습도, 기온, 풍속, 일사량
- 여러 데이터에서 필요한 정보들을 수집 후 최종 데이터 형태로 결합함
- 활용 데이터 목록
  - [환경 빅데이터 플랫폼_관측소별기상관측정보](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b1de30f0-38bb-11ea-be28-4fa0eb812a46)
  - [국가수자원관리종합정보시스템_실시간 기상자료](http://www.wamis.go.kr/wkw/we_dwtwtobs.do)
  - [농촌진흥청_기상통계](https://fruit.nihhs.go.kr/main/aws/awsStat.do)
  - [국립농업과학원_농업기상정보_주산지기상분석](http://weather.rda.go.kr/w/farmProduce/frcPlpdAvg.do)
  - [농업관측통합시스템](https://oasis.krei.re.kr/basicInfo/etc/kma.do)

### **📌 Data Description**
1. dam_nm: 댐이름
2. anc_dt: 관측일자
3. avg_hmd: 평균습도
4. avg_tmr: 평균기온
5. avg_wv: 평균풍속
6. snsn_sum: 일사량

### **a) 데이터 불러오기**

In [30]:
weather_path = '/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/관측소별기상관측정보' # path 변수에 경로 지정

In [31]:
weather_list = os.listdir(weather_path)
print(weather_list)
print(len(weather_list))

['일사량 결측치 핸들링.pdf', '202002.csv', '202003.csv', '202004.csv', '202005.csv', '202006.csv', '202008.csv', '202009.csv', '202010.csv', '202011.csv', '202101.csv', '202102.csv', '202103.csv', '202104.csv', '202105.csv', '202106.csv', '202207.csv', '202001.csv', '202007.csv', '202012.csv', '202107.csv', '202112.csv', '202202.csv', '202203.csv', '202204.csv', '202205.csv', '202206.csv', '202209.csv', '201901_201906.csv', '202108.csv', '202109.csv', '202201.csv', '201907_201912.csv', '202110.csv', '202111.csv', '202208.csv', '202211.csv', '202212.csv', '202210.csv']
39


In [32]:
df2 = file_concat(weather_path, weather_list)

In [33]:
df2

,dam_nm,anc_dt,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,군위,20200201,67.600000,2.900000,2.000000,7.100000
1,군위,20200202,61.500000,3.600000,1.200000,8.300000
2,군위,20200203,45.600000,3.100000,2.900000,7.200000
3,군위,20200204,42.500000,1.900000,1.800000,9.800000
4,군위,20200205,30.800000,-1.500000,4.500000,10.100000
...,...,...,...,...,...,...
30676,횡성,20221027,76.900000,10.100000,0.600000,7.000000
30677,횡성,20221028,75.900000,11.200000,0.700000,5.800000
30678,횡성,20221029,75.300000,11.100000,0.400000,6.100000
30679,횡성,20221030,75.800000,11.100000,0.500000,5.700000


In [34]:
### 데이터 정보 확인

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dam_nm    30681 non-null  object 
 1   anc_dt    30681 non-null  int64  
 2   avg_hmd   30620 non-null  float64
 3   avg_tmr   30676 non-null  float64
 4   avg_wv    30674 non-null  float64
 5   snsn_sum  30664 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


- 변수명 통일
  - 관측일자의 경우 다른 데이터와의 병합을 위해 ```anc_dt```에서 ```obsr_de```로 변경
- 데이터형 변경
  - obsr_de: object -> datetime
- 결측치 처리 필요

In [35]:
# 변수명 변경

df2.rename(columns={'anc_dt':'obsr_de'}, inplace = True)

In [36]:
df2

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,군위,20200201,67.600000,2.900000,2.000000,7.100000
1,군위,20200202,61.500000,3.600000,1.200000,8.300000
2,군위,20200203,45.600000,3.100000,2.900000,7.200000
3,군위,20200204,42.500000,1.900000,1.800000,9.800000
4,군위,20200205,30.800000,-1.500000,4.500000,10.100000
...,...,...,...,...,...,...
30676,횡성,20221027,76.900000,10.100000,0.600000,7.000000
30677,횡성,20221028,75.900000,11.200000,0.700000,5.800000
30678,횡성,20221029,75.300000,11.100000,0.400000,6.100000
30679,횡성,20221030,75.800000,11.100000,0.500000,5.700000


In [37]:
# 결측치나 변경할 수 없는 값이 있다면 일단 모두 무시하고 데이터형 변경

df2['obsr_de'] = df2['obsr_de'].apply(lambda x: pd.to_datetime(x, format = '%Y%m%d',errors = 'coerce'))

### **b) 이상치 처리**
- 기상 데이터에 대해서만 이상치 처리  
(일단은 수행 x)
- 이상치 제거를 수행하려면 주석 해제 후 이상치 제거 수행하시면 됩니당

In [38]:
### 이상치 탐지 함수

def get_outlier(df = None, dam_name = None, year = None, month = None, column = None, weight = 1.5):
    # 1/4 분위와 3/4 분위 지점을 np.percentile로 구함
    df_col = df[column]
    quantile_25 = np.percentile(df_col.values, 25)
    quantile_75 = np.percentile(df_col.values, 75)

    # IQR을 구하고, IQR에 1.5를 곱하여 최대값과 최소값 지점 구함.
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    # 최대값 보다 크거나, 최소값 보다 작은 값을 이상치로 설정하고 DataFrame index 반환
    outlier_index = df_col[(df_col < lowest_val) | (df_col > highest_val)].index

    # 이상치 정보를 저장할 dictionary 생성
    outlier_info = {
        'dam_nm': dam_name,
        'obsr_de': f'{year}년 {month}월',
        'column': column,
        'outlier_idx': outlier_index
    }

    return outlier_info

In [ ]:
# 댐 이름, 월, 컬럼명을 저장할 리스트 생성
outlier_info_list = []

cols = ['avg_hmd', 'avg_tmr', 'avg_wv', 'snsn_sum']
grouped = df2.groupby(['dam_nm', df['obsr_de'].dt.year, df['obsr_de'].dt.month])  # 댐별로 + 연도별 + 월별로

for col in cols:
  for (dam, year, month), group_data in grouped:
    outlier_info = get_outlier(group_data, dam_name=dam, year=year, month=month, column=col, weight=1.5)
    if len(outlier_info['outlier_idx']) > 0:
      outlier_info_list.append(outlier_info)

# 이상치 정보 출력
for outlier_info in outlier_info_list:
  dam_name = outlier_info['dam_nm']
  obsr_de = outlier_info['obsr_de']
  col = outlier_info['column']
  outlier_idx = outlier_info['outlier_idx']
  print('댐: {}, 월: {}, 컬럼: {}, 이상치 데이터 인덱스: {}'.format(dam_name, obsr_de, col, outlier_idx))
  print()

  # 이상치가 포함된 행 삭제
  # df.drop(outlier_idx, inplace = True) -- 일단은 수행 안함

In [40]:
# 이상치 처리 후 DataFrame 확인
# df

### **c) 결측치 처리**

In [41]:
# 현재 데이터 내의 결측치는 공백으로 처리되어 있음
# 데이터프레임 내의 공백을 NaN으로 변경

df2.replace('', np.nan, inplace = True)

In [42]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dam_nm    30681 non-null  object        
 1   obsr_de   30681 non-null  datetime64[ns]
 2   avg_hmd   30620 non-null  float64       
 3   avg_tmr   30676 non-null  float64       
 4   avg_wv    30674 non-null  float64       
 5   snsn_sum  30664 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.4+ MB


- 여러 변수들에 걸쳐 결측치가 있는 것으로 보인다.
- 결측치 처리
  - 습도 -> 최빈값
  - 온도 -> 평균
  - 풍속 -> 평균
  - 일사량 -> 최빈값

In [43]:
## 습도
len(df2.loc[df2['avg_hmd'].isna(),:])

61

In [44]:
# 해당 댐의 해당 연도 + 달의 습도 최빈값으로 대체

df2['avg_hmd'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['avg_hmd'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [45]:
df2.loc[df2['avg_hmd'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [46]:
## 온도

len(df2.loc[df2['avg_tmr'].isna(),:])

5

In [47]:
# 해당 댐의 해당 연도 + 달의 온도 평균값으로 대체

df2['avg_tmr'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['avg_tmr'].transform(lambda x: x.fillna(x.mean()))

In [48]:
df2.loc[df2['avg_tmr'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [49]:
## 풍속

len(df2.loc[df2['avg_wv'].isna(),:])

7

In [50]:
# 월별 계절 정보를 가지고 있는 함수를 정의합니다.
def get_season(month):
    if month in [3, 4, 5]:    # 봄
        return '봄'
    elif month in [6, 7, 8]:  # 여름
        return '여름'
    elif month in [9, 10, 11]:  # 가을
        return '가을'
    else:                     # 겨울 (12, 1, 2월)
        return '겨울'

In [ ]:
# 계절 정보 추가
df2['season'] = df2['obsr_de'].dt.month.apply(get_season)

# 여름과 겨울은 최빈값으로 대체, 봄과 가을은 평균값으로 대체
df2['avg_wv'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month, 'season'])['avg_wv'].transform(lambda x: x.fillna(x.mode()[0]) if x.name[2]=='여름' or x.name[2]=='겨울' else x.fillna(x.mean()))

In [52]:
df2.loc[df2['avg_wv'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum,season


In [53]:
# 계절 정보는 더이상 필요 없으므로 삭제

df2.drop('season', axis = 1, inplace = True)

In [54]:
## 일사량

len(df2.loc[df2['snsn_sum'].isna(),:])

17

In [55]:
# 해당 댐의 해당 연도 + 달의 습도 최빈값으로 대체

df2['snsn_sum'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['snsn_sum'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [56]:
df2.loc[df2['snsn_sum'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [57]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dam_nm    30681 non-null  object        
 1   obsr_de   30681 non-null  datetime64[ns]
 2   avg_hmd   30681 non-null  float64       
 3   avg_tmr   30681 non-null  float64       
 4   avg_wv    30681 non-null  float64       
 5   snsn_sum  30681 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.4+ MB


- 결측치가 제대로 채워졌다.

In [58]:
sum(~df['dam_nm'].isin(df2['dam_nm']))

0

- 댐 이름은 모두 동일

In [59]:
sum(~df['obsr_de'].isin(df2['obsr_de']))

0

- 관측일자도 모두 동일

In [60]:
duplicates2 = df2[df2.duplicated(subset = ['dam_nm', 'obsr_de'], keep=False)]
print(duplicates2)

Empty DataFrame
Columns: [dam_nm, obsr_de, avg_hmd, avg_tmr, avg_wv, snsn_sum]
Index: []


- 중복된 데이터는 없음을 확인할 수 있다.

In [61]:
### 데이터 병합

df = pd.merge(left = df, right = df2, how = "inner", on = ['dam_nm','obsr_de'] ) # dam_nm, 관측일 기준으로 조인

In [62]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt,valid_rsvwt_cpcty_qy,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,충주,2019-01-01,0.000000,3056918.400000,9040636.800000,2147765000.000000,78.100000,1789.000000,59.700000,-4.200000,1.700000,5.800000
1,충주,2019-01-02,0.000000,2209852.800000,9036748.800000,2140938000.000000,77.900000,1789.000000,57.600000,-6.400000,1.000000,10.100000
2,충주,2019-01-03,0.000000,1373932.800000,9039427.200000,2133272000.000000,77.600000,1789.000000,54.500000,-6.300000,0.800000,10.100000
3,충주,2019-01-04,0.000000,2237846.400000,9038476.800000,2126472000.000000,77.300000,1789.000000,57.700000,-4.200000,0.600000,6.700000
4,충주,2019-01-05,0.000000,2248300.800000,9036576.000000,2119683000.000000,77.100000,1789.000000,52.700000,-2.400000,1.100000,9.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
30676,장흥,2022-10-27,0.000000,0.000000,181267.200000,78686600.000000,41.200000,171.000000,67.800000,13.000000,0.800000,9.800000
30677,장흥,2022-10-28,0.000000,0.000000,181612.800000,78490900.000000,41.090000,171.000000,75.100000,12.800000,0.800000,9.800000
30678,장흥,2022-10-29,0.000000,0.000000,174441.600000,78295400.000000,40.990000,171.000000,77.800000,12.600000,0.700000,7.400000
30679,장흥,2022-10-30,0.000000,41990.400000,172195.200000,78165200.000000,40.920000,171.000000,78.600000,13.100000,0.900000,9.400000


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30681 entries, 0 to 30680
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   dam_nm                30681 non-null  object        
 1   obsr_de               30681 non-null  datetime64[ns]
 2   rain_qy               30681 non-null  float64       
 3   inflow_qy             30681 non-null  float64       
 4   tot_dcwtr_qy          30681 non-null  float64       
 5   rsvwt_qy              30681 non-null  float64       
 6   dam_rsvwt_rt          30681 non-null  float64       
 7   valid_rsvwt_cpcty_qy  30681 non-null  float64       
 8   avg_hmd               30681 non-null  float64       
 9   avg_tmr               30681 non-null  float64       
 10  avg_wv                30681 non-null  float64       
 11  snsn_sum              30681 non-null  float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 3.0+ MB


- 결측치 없이 잘 처리되었다.

In [64]:
df.to_csv("/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/병합 데이터(수빈)/final.csv")